# Data processing and dataset creation

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import tarfile
import os
import matplotlib.pyplot as plt

### Character metadata

In [3]:
header = ['Wikipedia movie ID', 'Freebase movie ID', 'Movie release date', 'Character name', 'Actor date of birth', "Actor gender","Actor height","Actor ethnicity","Actor name","Actor age at movie release","Freebase character/actor map ID", "Freebase character ID", "Freebase actor ID"]

df_characters = pd.read_csv('../../../MovieSummaries/character.metadata.tsv', delimiter='\t', encoding='utf-8', header=None, names=header)
# clean up the data
df_characters['Actor date of birth'] = pd.to_datetime(df_characters['Actor date of birth'], errors='coerce')
df_characters['Movie release date'] = pd.to_datetime(df_characters['Movie release date'], errors='coerce')
df_characters.head()

,Wikipedia movie ID,Freebase movie ID,Movie release date,Character name,Actor date of birth,Actor gender,Actor height,Actor ethnicity,Actor name,Actor age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26 00:00:00,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15 00:00:00,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15 00:00:00,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12 00:00:00,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25 00:00:00,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


### Movies Metadata

In [5]:
def extract_main_language(languages):
    # Function to extract the main language
    try:
        return languages.split(':')[1].split(',')[0].replace('"',"").replace("}","")
    except:
        return np.nan
    
def extract_main_genre(genres):
    # Function to extract the main genre
    try:
        return genres.split(':')[1].split(',')[0].replace('"',"").replace("}","")
    except:
        return np.nan
    
def keeping_most_common_genre(genre):

    ## This function does not work on the column of genres
    # Mapping of genres according to the 10 main categories
    genre_mapping = {
        "Thriller": ["Thriller"],
        "Drama": ["Drama", "Family Drama", "Melodrama", "Costume drama", "Tragedy"],
        "Crime": ["Crime Fiction"],
        "Short Film": ["Short Film"],
        "Romance": ["Romantic comedy", "Romance Film"],
        "Comedy": ["Comedy", "Parody", "Comedy film", "Comedy of Errors"],
        "Action/Adventure": ["Action/Adventure", "Action", "Adventure", "War film", "Swashbuckler films", "Martial Arts Film"],
        "Horror": ["Horror", "Monster movie", "Zombie Film", "Natural horror films"],
        "Science Fiction": ["Science Fiction", "Fantasy Adventure"],
        "Musical": ["Musical", "Music"]
    }

    for key, value in genre_mapping.items():
        if genre in value:
            return key
    return np.nan

print(keeping_most_common_genre("Mistery"))

    
def extract_main_country(countries):
    # Function to extract the main country
    try:
        return countries.split(':')[1].split(',')[0].replace('"',"").replace("}","")
    except:
        return np.nan

nan


In [6]:
header = ['Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 'Movie release date', 'Movie box office revenue', 'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres']
df_movie = pd.read_csv('../../../MovieSummaries/movie.metadata.tsv', delimiter='\t', encoding='utf-8', header=None, names=header)


df_movie["Main genre"] = df_movie["Movie genres"].apply(extract_main_genre)
#df_movie["Main genre 2"] = df_movie["Main genre"].apply(keeping_most_common_genre)
df_movie["Main language"] = df_movie['Movie languages'].apply(extract_main_language)
df_movie["Main country"] = df_movie['Movie countries'].apply(extract_main_country)

#df_movie['Main genre'].unique()
df_movie.head()

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,Main genre,Main language,Main country
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",Thriller,English Language,United States of America
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",Mystery,English Language,United States of America
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",Crime Fiction,Norwegian Language,Norway
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",Thriller,English Language,United Kingdom
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",Drama,German Language,Germany


### Cleaning textual data

In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

def clean_text(text):
    """
    Cleans the text by removing punctuation, stopwords, and lemmatizing the words.
    """
    # remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # remove stopwords
    text = [word for word in text.split() if word not in stopwords.words('english')]
    # lemmatize words
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text]
    return ' '.join(text)

[nltk_data] Downloading package stopwords to /Users/mathi/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/mathi/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /Users/mathi/nltk_data...


In [10]:
with open ('../../../MovieSummaries/plot_summaries.txt', 'r') as f:
    plot_summaries = f.readlines()

# keep the ID of the movie and the summary
plot_ids = [plot_summary.split('\t')[0] for plot_summary in plot_summaries]
plot_summaries = [plot_summary.split('\t')[1] for plot_summary in plot_summaries]

#clean the text
#plot_summaries_clean = [clean_text(plot_summary) for plot_summary in plot_summaries]
print(plot_summaries[0])
clean_text(plot_summaries[0])

Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.



'Shlykov hardworking taxi driver Lyosha saxophonist develop bizarre lovehate relationship despite prejudice realize arent different'

In [11]:
# Merge with the movies metadata dataframe

df_plot_summaries = pd.DataFrame({'Wikipedia movie ID': plot_ids, 'Plot summary': plot_summaries})
df_plot_summaries.head()

df_plot_summaries['Wikipedia movie ID'] = df_plot_summaries['Wikipedia movie ID'].astype(int)
df_movie['Wikipedia movie ID'] = df_movie['Wikipedia movie ID'].astype(int)

df_movie2 = df_movie.merge(df_plot_summaries, on='Wikipedia movie ID')

## Completing the dataset with the IMDb non-commercial dataset

In [13]:
df_ratings = pd.read_csv('../../../IMDb dataset/ratings.tsv', delimiter='\t', encoding='utf-8')

In [14]:
df_IMDb_movies = pd.read_csv('../../../IMDb dataset/title_basics.tsv', delimiter='\t', encoding='utf-8')


/var/folders/qd/996x_2t932b4mbltb5bx_y9r0000gn/T/ipykernel_2955/645384434.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_IMDb_movies = pd.read_csv('../../../IMDb dataset/title_basics.tsv', delimiter='\t', encoding='utf-8')


In [18]:
df_IMDb_movies2 = df_IMDb_movies.merge(df_ratings, on = 'tconst', how = 'left').reset_index(drop = True)

### Data cleaning

In [19]:
df_IMDb_movies2["startYear"] = pd.to_numeric(df_IMDb_movies2["startYear"], errors='coerce')
df_IMDb_movies2 = df_IMDb_movies2[df_IMDb_movies2['startYear'] > 1920] # keeping only movies after 1920
df_IMDb_movies2 = df_IMDb_movies2[df_IMDb_movies2['titleType'].isin(['movie', 'short'])] # keep only movies and short
df_IMDb_movies2 = df_IMDb_movies2[~df_IMDb_movies2['averageRating'].isna()].reset_index(drop = True) # keep only movies with ratings: passes from 13M movies to 400k movies

In [20]:
df_IMDb_movies2["primaryTitle_lower"] = df_IMDb_movies2["primaryTitle"].str.lower()
df_IMDb_movies2["originalTitle_lower"] = df_IMDb_movies2["originalTitle"].str.lower()

### Merging with df_movie

In [21]:
def clean_release_date(date):
    try:
        return int(date[0:4])
    except:
        return np.nan   
    
df_movie2['Release Date'] = df_movie2['Movie release date'].apply(clean_release_date)
df_movie2['title_lower'] = df_movie2['Movie name'].str.lower()

In [22]:
# Merge the DataFrames based on name and date
df_movie3 = df_movie2.merge(df_IMDb_movies2, left_on=['title_lower', 'Release Date'], right_on=['primaryTitle_lower', 'startYear'], how='left')

# Reset the index
df_movie3 = df_movie3.reset_index(drop=True)

In [23]:
# proportion of movies with IMBD ratings
df_movie3[~df_movie3['tconst'].isna()].shape[0] / df_movie3.shape[0]*100 

63.169104739613815

In [24]:
# We only keep the columns we are interested in

df_movies_columns_keep = ['Movie name', 'Release Date', 'Movie box office revenue', 'Movie runtime', 'Main genre',
       'Main language', 'Main country', 'Plot summary', 'tconst', 'averageRating', 'numVotes']

df_movies = df_movie3[df_movies_columns_keep]
df_movies.head()

,Movie name,Release Date,Movie box office revenue,Movie runtime,Main genre,Main language,Main country,Plot summary,tconst,averageRating,numVotes
0,Ghosts of Mars,2001.0,14010832.0,98.0,Thriller,English Language,United States of America,"Set in the second half of the 22nd century, th...",tt0228333,4.9,56894.0
1,White Of The Eye,1987.0,NaN,110.0,Thriller,English Language,United Kingdom,A series of murders of rich young women throug...,tt0094320,6.1,2894.0
2,A Woman in Flames,1983.0,NaN,106.0,Drama,German Language,Germany,"Eva, an upper class housewife, becomes frustra...",tt0083949,5.9,623.0
3,The Sorcerer's Apprentice,2002.0,NaN,86.0,Family Film,English Language,South Africa,"Every hundred years, the evil Morgana returns...",NaN,NaN,NaN
4,Little city,1997.0,NaN,93.0,Romantic comedy,English Language,United States of America,"Adam, a San Francisco-based artist who works a...",tt0119548,5.8,1129.0


In [ ]:
## Adding budget data

In [31]:
with open('../../../movie_data_tmdb.csv', 'r') as file:
    lines = file.readlines()

# Split each line using the '|' character as the delimiter
data = [line.strip().split('|') for line in lines]

# Create a DataFrame from the split data
df = pd.DataFrame(data)
df.rename(columns={0: 'adult', 1: 'title', 2: 'release_date', 3: 'budget', 4: 'revenue', 5: 'runtime', 6: 'genres', 7: 'IMDb', 8: 'vote_count'}, inplace=True)
df = df[['budget', 'IMDb']]
merged = df_movies.merge(df, left_on='tconst', right_on= 'IMDb', how='left').drop('IMDb', axis=1).drop_duplicates()


In [32]:
display(merged.head())
print(merged.shape)

,Movie name,Release Date,Movie box office revenue,Movie runtime,Main genre,Main language,Main country,Plot summary,tconst,averageRating,numVotes,budget
0,Ghosts of Mars,2001.0,14010832.0,98.0,Thriller,English Language,United States of America,"Set in the second half of the 22nd century, th...",tt0228333,4.9,56894.0,28000000
1,White Of The Eye,1987.0,NaN,110.0,Thriller,English Language,United Kingdom,A series of murders of rich young women throug...,tt0094320,6.1,2894.0,0
2,A Woman in Flames,1983.0,NaN,106.0,Drama,German Language,Germany,"Eva, an upper class housewife, becomes frustra...",tt0083949,5.9,623.0,0
3,The Sorcerer's Apprentice,2002.0,NaN,86.0,Family Film,English Language,South Africa,"Every hundred years, the evil Morgana returns...",NaN,NaN,NaN,None
657,Little city,1997.0,NaN,93.0,Romantic comedy,English Language,United States of America,"Adam, a San Francisco-based artist who works a...",tt0119548,5.8,1129.0,0


(42725, 12)


## Adding Oscars wins and nominations

In [39]:
oscars = pd.read_csv('../datasets/oscar.csv')
oscars = oscars[['Film', 'Year', 'Award', 'Nomination']]
oscars = oscars[(oscars['Year'] <= 2014) & (oscars['Year'] != 21)] # 21 is a typo and our metadata dataset stops in 2014
display(oscars.head())

,Film,Year,Award,Nomination
121,Birdman or (The Unexpected Virtue of Ignorance),2014,4,9
122,The Grand Budapest Hotel,2014,4,9
123,Whiplash,2014,3,5
124,The Imitation Game,2014,1,8
125,American Sniper,2014,1,6


In [41]:
merged2 = merged.merge(oscars, left_on=['Movie name', 'Release Date'], right_on= ['Film', 'Year'], how='left').drop(['Film', 'Year'], axis=1)
merged2['Award'].fillna(0, inplace=True)
merged2['Award'] = merged2['Award'].astype(int)
merged2['Nomination'].fillna(0, inplace=True)
merged2['Nomination'] = merged2['Nomination'].astype(int)
merged2.rename(columns={'Award': 'Oscar Wins', 'Nomination': 'Nominations', 'budget' : 'Estimated Budget'}, inplace=True)
display(merged2.sort_values(by='Nominations', ascending=False).head())

,Movie name,Release Date,Movie box office revenue,Movie runtime,Main genre,Main language,Main country,Plot summary,tconst,averageRating,numVotes,Estimated Budget,Oscar Wins,Nominations
17749,All About Eve,1950.0,2.900000e+06,139.0,LGBT,French Language,United States of America,"At an awards dinner, Eve Harrington &nbsp;— t...",tt0042192,8.2,136369.0,1400000,6,14
11653,Titanic,1997.0,2.185372e+09,194.0,Tragedy,Italian Language,United States of America,"In 1996, treasure hunter Brock Lovett and his...",tt0120338,7.9,1252142.0,200000000,11,14
8,Mary Poppins,1964.0,1.022727e+08,139.0,Children's/Family,English Language,United States of America,The film opens with Mary Poppins perched in a...,tt0058331,7.8,181710.0,6000000,5,13
17420,From Here to Eternity,1953.0,3.050000e+07,118.0,War film,English Language,United States of America,"In 1941, Private Robert E. Lee Prewitt , a bug...",tt0045793,7.6,49880.0,1650000,8,13
19807,Gone with the Wind,1939.0,4.000000e+08,234.0,Film adaptation,English Language,United States of America,The film opens on a large cotton plantation c...,tt0031381,8.2,327988.0,4000000,8,13


## Adding inflation adjusted profit data

In [42]:
df = pd.read_csv('../datasets/US CPI.csv')
display(df.head())

,Yearmon,CPI
0,01-01-1913,9.8
1,01-02-1913,9.8
2,01-03-1913,9.8
3,01-04-1913,9.8
4,01-05-1913,9.7


In [44]:
# We have the CPI per month but want it per year, thus we need to be able to apply a group by year
# and need the first column to only contain the year

df['Yearmon'] = df['Yearmon'].astype(str)

mask = df['Yearmon'].str.contains('|'.join(map(str, range(1913, 2024))))

df.loc[mask, 'Yearmon'] = df.loc[mask, 'Yearmon'].str.extract(f'({"|".join(map(str, range(1913, 2024)))})')[0]

print(df.head())

  Yearmon  CPI
0    1913  9.8
1    1913  9.8
2    1913  9.8
3    1913  9.8
4    1913  9.7


In [50]:
# We get the CPI per year by averaging the CPI over the 12 months

df = df.rename(columns={'Yearmon' : 'Year'})
df = df.groupby('Year')['CPI'].mean().reset_index()
df['Year'] = df['Year'].astype(int)
df = df[df['Year'] <= 2014]
display(df)  


,Year,CPI
0,1913,9.883333
1,1914,10.016667
2,1915,10.108333
3,1916,10.883333
4,1917,12.825000
...,...,...
97,2010,218.055500
98,2011,224.939167
99,2012,229.593917
100,2013,232.957083


### Computing Inflation rate based on CPI data 

In [51]:
target_year = 2014  # Replace with the desired year

# Get the CPI for the target year
cpi_2023 = df[df['Year'] == target_year]['CPI'].values[0]

# Calculate the inflation rate for each year compared to 2023
df['Inflation Rate'] = (cpi_2023 - df['CPI']) / df['CPI']

display(df)

,Year,CPI,Inflation Rate
0,1913,9.883333,22.953069
1,1914,10.016667,22.634226
2,1915,10.108333,22.419901
3,1916,10.883333,20.752175
4,1917,12.825000,17.458960
...,...,...,...
97,2010,218.055500,0.085669
98,2011,224.939167,0.052445
99,2012,229.593917,0.031108
100,2013,232.957083,0.016222


### Adding the profit inflation adjusted to 2014 value and the inflation rate used for every movie 

In [52]:
merged2['Estimated Budget'] = merged2['Estimated Budget'].fillna(0)
merged2['Estimated Budget'] = merged2['Estimated Budget'].astype(int)
merged2['Movie box office revenue'] = merged2['Movie box office revenue'].fillna(0)
merged2['Movie box office revenue'] = merged2['Movie box office revenue'].astype(int)
merged2['Profit'] = merged2['Movie box office revenue'] - merged2['Estimated Budget']
merged2['Release Date'] = merged2['Release Date'].fillna(0).astype(int)


In [59]:
merged3 = merged2.merge(df, left_on='Release Date', right_on= 'Year', how='left').drop(['Year', 'CPI'], axis=1)
merged3['Inflation adjusted profit'] = merged3['Profit'] * (1 + merged3['Inflation Rate'])
display(merged3.sort_values(by='Inflation adjusted profit', ascending=False).head(10))

,Movie name,Release Date,Movie box office revenue,Movie runtime,Main genre,Main language,Main country,Plot summary,tconst,averageRating,numVotes,Estimated Budget,Oscar Wins,Nominations,Profit,Inflation Rate,Inflation adjusted profit
6322,Snow White and the Seven Dwarfs,1937,416000000,84.0,Romance Film,English Language,United States of America,Through a textual prologue told via a storybo...,tt0029583,7.6,211082.0,1488423,0,1,414511577,15.459061,6.822472e+09
19807,Gone with the Wind,1939,400000000,234.0,Film adaptation,English Language,United States of America,The film opens on a large cotton plantation c...,tt0031381,8.2,327988.0,4000000,8,13,396000000,16.021174,6.740385e+09
673,Bambi,1942,267447150,69.0,Adventure,English Language,United States of America,"A doe gives birth to a fawn named Bambi, who w...",tt0034492,7.3,150961.0,858000,0,0,266589150,13.494051,3.863957e+09
3728,Love with the Proper Stranger,1963,415004880,121.0,Social problem film,English Language,United States of America,"The film tells the story of Angie Rossini , a ...",tt0057263,7.3,5325.0,8500000,0,0,406504880,6.730161,3.142348e+09
42234,Star Wars Episode IV: A New Hope,1977,775398007,122.0,Science Fiction,English Language,United States of America,The film begins with an opening crawl explain...,NaN,NaN,NaN,0,0,0,775398007,2.906000,3.028705e+09
11653,Titanic,1997,2185372302,194.0,Tragedy,Italian Language,United States of America,"In 1996, treasure hunter Brock Lovett and his...",tt0120338,7.9,1252142.0,200000000,11,14,1985372302,0.474839,2.928104e+09
12410,Avatar,2009,2782275172,178.0,Thriller,English Language,United States of America,"By 2154, humans have severely depleted Earth'...",tt0499549,7.9,1364377.0,237000000,3,9,2545275172,0.103475,2.808647e+09
12368,The Exorcist,1973,441071011,120.0,Demonic child,Arabic Language,United States of America,"At an archaeological dig in Northern Iraq, arc...",tt0070047,8.1,443245.0,8000000,2,10,433071011,4.331896,2.309089e+09
22851,The Sound of Music,1965,286214286,172.0,Children's/Family,English Language,United States of America,"{{Plot}} Maria is found in a pasture, exultin...",tt0059742,8.1,251011.0,8200000,5,10,278014286,6.513446,2.088845e+09
40557,Jaws,1975,470653000,130.0,Thriller,English Language,United States of America,A girl named Chrissie Watkins leaves an eveni...,tt0073195,8.1,644272.0,7000000,3,4,463653000,3.398938,2.039581e+09


In [60]:
# We drop the inflation rate column as we do not need it anymore
merged3 = merged3.drop(columns = 'Inflation Rate', axis=1)
display(merged3.head())

,Movie name,Release Date,Movie box office revenue,Movie runtime,Main genre,Main language,Main country,Plot summary,tconst,averageRating,numVotes,Estimated Budget,Oscar Wins,Nominations,Profit,Inflation adjusted profit
0,Ghosts of Mars,2001,14010832,98.0,Thriller,English Language,United States of America,"Set in the second half of the 22nd century, th...",tt0228333,4.9,56894.0,28000000,0,0,-13989168,-1.870336e+07
1,White Of The Eye,1987,0,110.0,Thriller,English Language,United Kingdom,A series of murders of rich young women throug...,tt0094320,6.1,2894.0,0,0,0,0,0.000000e+00
2,A Woman in Flames,1983,0,106.0,Drama,German Language,Germany,"Eva, an upper class housewife, becomes frustra...",tt0083949,5.9,623.0,0,0,0,0,0.000000e+00
3,The Sorcerer's Apprentice,2002,0,86.0,Family Film,English Language,South Africa,"Every hundred years, the evil Morgana returns...",NaN,NaN,NaN,0,0,0,0,0.000000e+00
4,Little city,1997,0,93.0,Romantic comedy,English Language,United States of America,"Adam, a San Francisco-based artist who works a...",tt0119548,5.8,1129.0,0,0,0,0,0.000000e+00


# Directors data (Director dataset + adding director name and id to main dataframe)

In [70]:
title_crew = pd.read_csv("../../../IMDb dataset/crew.tsv", sep='\t')
title_crew = title_crew.drop(columns=['writers'])

In [71]:
crew_name = pd.read_csv("../../../IMDb dataset/names.tsv", sep='\t')
crew_name = crew_name.drop(columns=['primaryProfession', 'knownForTitles'])
display(crew_name.head())

,nconst,primaryName,birthYear,deathYear
0,nm0000001,Fred Astaire,1899,1987
1,nm0000002,Lauren Bacall,1924,2014
2,nm0000003,Brigitte Bardot,1934,\N
3,nm0000004,John Belushi,1949,1982
4,nm0000005,Ingmar Bergman,1918,2007


## Creating a dataframe with data for each director

In [72]:
# When a title has multiple directors, we keep the first one
title_crew['directors'] = title_crew['directors'].fillna('')
title_crew['directors'] = title_crew['directors'].apply(lambda x: x.split(','))
title_crew['directors'] = title_crew['directors'].apply(lambda x: x[0])


In [73]:
display(title_crew.head())

,tconst,directors
0,tt0000001,nm0005690
1,tt0000002,nm0721526
2,tt0000003,nm0721526
3,tt0000004,nm0721526
4,tt0000005,nm0005690


In [74]:
# We add the name and birth and death dates to the crew per title dataframe
title_crew2 = title_crew.merge(crew_name, left_on='directors', right_on='nconst', how='left').drop(['directors'], axis=1)
display(title_crew2.head())

,tconst,nconst,primaryName,birthYear,deathYear
0,tt0000001,nm0005690,William K.L. Dickson,1860,1935
1,tt0000002,nm0721526,Émile Reynaud,1844,1918
2,tt0000003,nm0721526,Émile Reynaud,1844,1918
3,tt0000004,nm0721526,Émile Reynaud,1844,1918
4,tt0000005,nm0005690,William K.L. Dickson,1860,1935


In [75]:
# We create a new dataframe where we have for each director its nconst, number of movies directed, the list of the 
# tconst of the directed movies, its birth and death year
director_df = title_crew2.groupby('nconst').agg(
    director_name=('primaryName', 'first'),
    num_movies=('tconst', 'count'),
    movies=('tconst', lambda x: list(x)),
    birth_year=('birthYear', 'first'),
    death_year=('deathYear', 'first')
).reset_index()
display(director_df.head())

,nconst,director_name,num_movies,movies,birth_year,death_year
0,nm0000005,Ingmar Bergman,85,"[tt0038468, tt0038675, tt0039834, tt0040418, t...",1918,2007
1,nm0000008,Marlon Brando,1,[tt0055257],1924,2004
2,nm0000009,Richard Burton,1,[tt0062898],1925,1984
3,nm0000010,James Cagney,1,[tt0050964],1899,1986
4,nm0000018,Kirk Douglas,1,[tt0073559],1916,2020


## Adding the director data to the main dataframe

In [76]:
merged4 = merged3.merge(title_crew2[['tconst', 'nconst', 'primaryName']], left_on='tconst', right_on='tconst', how='left')
merged4 = merged4.rename(columns={'primaryName': 'Director name'})
merged4['Director name'].fillna('Unknown', inplace=True)
display(merged4.head())


,Movie name,Release Date,Movie box office revenue,Movie runtime,Main genre,Main language,Main country,Plot summary,tconst,averageRating,numVotes,Estimated Budget,Oscar Wins,Nominations,Profit,Inflation adjusted profit,nconst,Director name
0,Ghosts of Mars,2001,14010832,98.0,Thriller,English Language,United States of America,"Set in the second half of the 22nd century, th...",tt0228333,4.9,56894.0,28000000,0,0,-13989168,-1.870336e+07,nm0000118,John Carpenter
1,White Of The Eye,1987,0,110.0,Thriller,English Language,United Kingdom,A series of murders of rich young women throug...,tt0094320,6.1,2894.0,0,0,0,0,0.000000e+00,nm0131910,Donald Cammell
2,A Woman in Flames,1983,0,106.0,Drama,German Language,Germany,"Eva, an upper class housewife, becomes frustra...",tt0083949,5.9,623.0,0,0,0,0,0.000000e+00,nm0885554,Robert van Ackeren
3,The Sorcerer's Apprentice,2002,0,86.0,Family Film,English Language,South Africa,"Every hundred years, the evil Morgana returns...",NaN,NaN,NaN,0,0,0,0,0.000000e+00,NaN,Unknown
4,Little city,1997,0,93.0,Romantic comedy,English Language,United States of America,"Adam, a San Francisco-based artist who works a...",tt0119548,5.8,1129.0,0,0,0,0,0.000000e+00,nm0070142,Roberto Benabib


# Actors data (Four principal roles for every movie and actor name and age)

In [77]:
principal_actors = pd.read_csv("../../../IMDb dataset/title_principals.tsv", sep='\t')
principal_actors = principal_actors[(principal_actors['category'] == 'actor') | (principal_actors['category'] == 'actress')]
principal_actors2 = principal_actors.merge(crew_name, left_on='nconst', right_on='nconst', how='left').drop(columns = ['job'])

In [79]:
# only TV series have more than 4 principal roles so we get rid of them

tconst_with_ordering_ge_5 = principal_actors2.loc[principal_actors2['ordering'] >= 5, 'tconst'].values 
principal_actors2 = principal_actors2[~principal_actors2['tconst'].isin(tconst_with_ordering_ge_5)]


In [81]:
display(principal_actors2.head(15))

,tconst,ordering,nconst,category,characters,primaryName,birthYear,deathYear
0,tt0000005,1,nm0443482,actor,"[""Blacksmith""]",Charles Kayser,1878,1966
1,tt0000005,2,nm0653042,actor,"[""Assistant""]",John Ott,1850,1931
2,tt0000007,1,nm0179163,actor,\N,James J. Corbett,1866,1933
3,tt0000007,2,nm0183947,actor,\N,Peter Courtney,1867,1896
4,tt0000008,1,nm0653028,actor,"[""Sneezing Man""]",Fred Ott,1860,1936
5,tt0000009,1,nm0063086,actress,"[""Miss Geraldine Holbrook (Miss Jerry)""]",Blanche Bayliss,1878,1951
6,tt0000009,2,nm0183823,actor,"[""Mr. Hamilton""]",William Courtenay,1875,1933
7,tt0000009,3,nm1309758,actor,"[""Chauncey Depew - the Director of the New Yor...",Chauncey Depew,1834,1928
8,tt0000011,1,nm3692297,actor,"[""Acrobats""]",Grunato,\N,\N
9,tt0000014,1,nm0166380,actor,"[""The Gardener""]",François Clerc,\N,\N


In [80]:
print("We have a dataset of {} directors.".format(f"{director_df.shape[0]:,}"))
print("We have a dataset of {} principal roles.".format(f"{principal_actors2.shape[0]:,}"))

We have a dataset of 686,042 directors.
We have a dataset of 12,181,812 principal roles.
